# Generate Features

In [69]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from utils.import_utils import *
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from omegaconf import OmegaConf
with initialize(config_path='../configuration', version_base='1.1'):
    config = compose(config_name='main.yaml')
survey_list = SurveyManager(config)
dfs_paradata, dfs_questionnaires, dfs_microdata = survey_list.get_dataframes(reload=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/Users/andreas/projects/mlss/venv/lib/python3.10/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'main.yaml': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)


In [58]:
import numpy as np
#group_columns = [col for col in dfs_microdata.columns if col.endswith("__id")]+['survey_name', 'survey_version']
feat_item = dfs_microdata[['variable', 'value', 'type', 'IsInteger', 'n_answers', 'answer_sequence']].copy()

feat_item['value'].fillna('', inplace=True)

text_question_mask = (feat_item['type'] == 'TextQuestion')
numeric_question_mask = (feat_item['type'] == 'NumericQuestion') & (feat_item['value'] != '')
decimal_question_mask = (feat_item['IsInteger'] == False) & (feat_item['value'] != '')


In [59]:
# f__string_length, length of string answer, if TextQuestions, empty if not
feat_item['f__string_length'] = ''
feat_item.loc[text_question_mask, 'f__string_length'] = feat_item.loc[text_question_mask, 'value'].str.len()

In [60]:
# f__first_digit, first digit of the response if numeric question, empty if not
feat_item['f__first_digit'] = np.nan
feat_item.loc[numeric_question_mask, 'f__first_digit'] = feat_item.loc[numeric_question_mask, 'value'].astype(str).str[0]

In [61]:
# f__last_digit, modulus of 10 of the response if numeric question, empty if not
feat_item['f__last_digit'] = np.nan
feat_item.loc[numeric_question_mask, 'f__last_digit'] = feat_item.loc[numeric_question_mask, 'value'].astype(float) % 10

In [71]:
# f__first_decimal, first decimal digit if numeric question, empty if not
feat_item['f__first_decimal'] = np.nan
values = feat_item.loc[decimal_question_mask, 'value'].astype(float)
feat_item.loc[decimal_question_mask, 'f__first_decimal'] = np.floor(values * 10) % 10
feat_item['f__first_decimal']=feat_item['f__first_decimal'].astype('Int64')

In [68]:
# f__rel_answer_position, relative position of the selected answer
feat_item['f__rel_answer_position'] = np.nan
single_question_mask = feat_item['n_answers'] > 2 # only questions with more than two answers
feat_item.loc[single_question_mask, 'f__answer_position'] = feat_item.loc[single_question_mask].apply(lambda row: row['answer_sequence'].index(int(row['value'])) if (row['value'] in row['answer_sequence']) and pd.notnull(row['value']) else None, axis=1)


# Questions to Gabriele

1. how do we deal with the nan, so we see if the item contains a response or not? nan for other question types and if there is no value
2. instead of renaming the question type, rather work with the variable instead, less change, no?
3.

In [37]:
import json
def parse_json(x):
    try:
        return json.loads(x.replace("'", "\""))
    except:
        return []
dfs_questionnaires['Answers'] = dfs_questionnaires['Answers'].apply(parse_json)
dfs_questionnaires['Answers']


0                                                    []
1                                                    []
2                                                    []
3                                                    []
4     [{'AnswerText': 'Medenine', 'AnswerValue': '1'...
                            ...                        
67    [{'AnswerText': 'Oui', 'AnswerValue': '1'}, {'...
68                                                   []
69    [{'AnswerText': 'Oui', 'AnswerValue': '1'}, {'...
70                                                   []
71    [{'AnswerText': 'Très satisfait', 'AnswerValue...
Name: Answers, Length: 482, dtype: object

In [47]:

# Create new columns
dfs_questionnaires['answer_sequence'] = dfs_questionnaires['Answers'].apply(lambda x: [item['AnswerValue'] for item in x] if x else np.nan)
dfs_questionnaires['n_answers'] = dfs_questionnaires['Answers'].apply(lambda x: len(x) if x else np.nan)



In [48]:
dfs_questionnaires

,question_seq,VariableName,type,QuestionType,Answers,Children,ConditionExpression,HideIfDisabled,Featured,Instructions,...,parent_1,parent_2,parent_3,parent_4,survey_name,survey_version,first_answer,last_answer,n_answers,answer_sequence
0,1,NaN,Group,NaN,[],"[{'$type': 'TextQuestion', 'QuestionType': 7, ...",NaN,False,NaN,NaN,...,NaN,NaN,NaN,NaN,ifad_tunesia,EndlineFINALV106_1,NaN,NaN,NaN,NaN
1,2,country,TextQuestion,7.0,[],[],NaN,False,True,NaN,...,COVER,NaN,NaN,NaN,ifad_tunesia,EndlineFINALV106_1,NaN,NaN,NaN,NaN
2,3,NaN,Group,NaN,[],"[{'$type': 'DateTimeQuestion', 'QuestionType':...",NaN,False,NaN,NaN,...,NaN,NaN,NaN,NaN,ifad_tunesia,EndlineFINALV106_1,NaN,NaN,NaN,NaN
3,4,start_visitdate,DateTimeQuestion,5.0,[],[],NaN,False,False,NaN,...,BASIC INFORMATION AND CONSENT,NaN,NaN,NaN,ifad_tunesia,EndlineFINALV106_1,NaN,NaN,NaN,NaN
4,5,region,SingleQuestion,0.0,"[{'AnswerText': 'Medenine', 'AnswerValue': '1'...",[],NaN,False,False,NaN,...,BASIC INFORMATION AND CONSENT,NaN,NaN,NaN,ifad_tunesia,EndlineFINALV106_1,1,2,2.0,"[1, 2]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,68,fromation_use,SingleQuestion,0.0,"[{'AnswerText': 'Oui', 'AnswerValue': '1'}, {'...",[],NaN,False,False,NaN,...,J - PO,PO FORMATION,PO FORMATION - %rostertitle%,NaN,ifad_tunesia,EndlineFINALV5_1,1,2,3.0,"[1, 0, 2]"
68,69,NaN,Group,NaN,[],"[{'$type': 'SingleQuestion', 'QuestionType': 0...",NaN,False,NaN,NaN,...,J - PO,NaN,NaN,NaN,ifad_tunesia,EndlineFINALV5_1,NaN,NaN,NaN,NaN
69,70,partner,SingleQuestion,0.0,"[{'AnswerText': 'Oui', 'AnswerValue': '1'}, {'...",[],NaN,False,False,Par example,...,J - PO,OP PARTERNSHIPS,NaN,NaN,ifad_tunesia,EndlineFINALV5_1,1,0,2.0,"[1, 0]"
70,71,partner_number,NumericQuestion,4.0,[],[],partner==1,False,False,NaN,...,J - PO,OP PARTERNSHIPS,NaN,NaN,ifad_tunesia,EndlineFINALV5_1,NaN,NaN,NaN,NaN
